In [12]:
import cv2
import pyrealsense2 as rs
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt

img_height = 120
img_width = 160
PERCENT = 25

model = keras.Sequential(
        [
            layers.Conv2D(128, kernel_size=(3, 4), input_shape=(120, 160, 3), strides=(1, 1), padding='valid',
                          activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(64, 3, padding="same", activation="relu"),
            layers.Conv2D(32, 3, padding="same", activation="relu"),
            layers.MaxPooling2D(),
            layers.BatchNormalization(),
            layers.Conv2D(32, 3, padding="same", activation="relu"),
            layers.MaxPooling2D(),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dense(200, activation='relu'),
            layers.Dense(100, activation='relu'),
            layers.Dropout(0.4),
            layers.Dense(10, activation='softmax'),
        ]
    )

colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]
actions = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])


def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
    return output_frame


def resize_image(img):
    width = int(img.shape[1] * PERCENT / 100)
    height = int(img.shape[0] * PERCENT / 100)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    return resized

In [13]:
model.load_weights('rgb_only_new_weights.h5')

In [14]:
img = cv2.imread("2021-09-21-122947_Color.png")
arr = np.array(resize_image(img))
arr.shape

(120, 160, 3)

In [22]:
arr = arr.reshape(1, arr.shape[0], arr.shape[1], arr.shape[2])
arr.shape

ValueError: cannot reshape array of size 57600 into shape (1,1,120,160)

In [20]:
model.predict(arr)

array([[0.00976823, 0.07881062, 0.07637336, 0.774524  , 0.02231396,
        0.01900523, 0.00139486, 0.01109907, 0.0020866 , 0.00462417]],
      dtype=float32)

In [4]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

# Setup pipeline
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)


while True:

    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    if not depth_frame or not color_frame:
        break
    colorizer = rs.colorizer()
    colorizer.set_option(rs.option.color_scheme, 0)
    # Convert images to numpy arrays
    image = np.asanyarray(color_frame.get_data())
    print(image.shape)
    res = model.predict(image, batch_size=1)
    print(actions[np.argmax(res)])
    predictions.append(np.argmax(res))

    # 3. Viz logic
    if np.unique(predictions[-10:])[0] == np.argmax(res):
        if res[np.argmax(res)] > threshold:

            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])

    if len(sentence) > 5:
        sentence = sentence[-5:]

    # Viz probabilities
    image = prob_viz(res, actions, image, colors)

    cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
    cv2.putText(image, ' '.join(sentence), (3, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Show to screen
    cv2.imshow('OpenCV Feed', image)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
pipeline.stop()

(480, 640, 3)


ValueError: in user code:

    c:\python38\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    c:\python38\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\python38\lib\site-packages\keras\engine\training.py:1569 run_step  **
        outputs = model.predict_step(data)
    c:\python38\lib\site-packages\keras\engine\training.py:1537 predict_step
        

In [9]:
image = cv2.imread('2021-09-21-122947_Color.png')
image = np.asanyarray(image)
print(image.shape)
#model.call(tf.convert_to_tensor(image))

(480, 640, 3)


ValueError: Input 0 of layer conv2d_3 is incompatible with the layer: : expected min_ndim=4, found ndim=3. Full shape received: (480, 640, 3)

In [16]:
loaded_model = tf.keras.models.load_model('rgb_only_weights.h5')
loaded_model.layers[0].input_shape()
image_path="2021-09-21-122947_Color.png"
img = image.load_img(image_path, target_size=(160, 160))
plt.imshow(img)
img = np.expand_dims(img, axis=0)
result=loaded_model.predict_classes(img)
plt.title(actions(result[0][0]))
plt.show()

TypeError: 'tuple' object is not callable